In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations, CsvParser

from common import SHARED_STORE, SNOBAL_DIR, SNOTEL_DIR, DATA_DIR, COARSEN_OPTS, HV_PLOT_OPTS

In [ ]:
client = start_cluster(6, 24)
client_ip_and_port(client)

In [ ]:
water_year = 'wy2021'

## SNOTEL 

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites.json')

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Thermal W/m^2'

# HRRR

## Thermal 

In [ ]:
HRRR_therm = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/smrf_20*.nc',
    parallel=True, chunks={'time': 744, 'y' :1000, 'x': 1000},
    drop_variables=['precip_temp', 'percent_snow', 'precip', 'snow_density', 'storm_days', 'wind_speed', 'vapor_pressure']
)

butte_snobal_hrrr = HRRR_therm.thermal.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').squeeze(['x', 'y']).compute()
schofield_snobal_hrrr = HRRR_therm.thermal.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').squeeze(['x', 'y']).compute()
taylor_snobal_hrrr = HRRR_therm.thermal.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').squeeze(['x', 'y']).compute()
irwin_snobal_hrrr = HRRR_therm.thermal.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').squeeze(['x', 'y']).compute()

## Clouds 

In [ ]:
HRRR_therm = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/cloud_factor.nc',
    parallel=True, chunks={'time': 744, 'y' :100, 'x': 100},
)
butte_snobal_hrrr_cloud = HRRR_therm.TCDC.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').squeeze(['x', 'y']).compute()
schofield_snobal_hrrr_cloud = HRRR_therm.TCDC.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').squeeze(['x', 'y']).compute()
taylor_snobal_hrrr_cloud = HRRR_therm.TCDC.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').squeeze(['x', 'y']).compute()
irwin_snobal_hrrr_cloud = HRRR_therm.TCDC.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').squeeze(['x', 'y']).compute()

del HRRR_therm

# SMRF 

In [ ]:
smrf_out = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw/run*/smrf_20*.nc',
    parallel=True, chunks={'time': 744, 'y' :100, 'x': 100},
    drop_variables=['precip_temp', 'percent_snow', 'precip', 'snow_density', 'storm_days', 'wind_speed', 'vapor_pressure']
)

## Thermal

In [ ]:
butte_snobal = smrf_out.thermal.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').squeeze(['x', 'y']).compute()
schofield_snobal = smrf_out.thermal.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').squeeze(['x', 'y']).compute()
taylor_snobal = smrf_out.thermal.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').squeeze(['x', 'y']).compute()
irwin_snobal = smrf_out.thermal.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').squeeze(['x', 'y']).compute()

## Clouds 

In [ ]:
butte_snobal_cloud = smrf_out.cloud_factor.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').squeeze(['x', 'y']).compute()
schofield_snobal_cloud = smrf_out.cloud_factor.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').squeeze(['x', 'y']).compute()
taylor_snobal_cloud = smrf_out.cloud_factor.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').squeeze(['x', 'y']).compute()
irwin_snobal_cloud = smrf_out.cloud_factor.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').squeeze(['x', 'y']).compute()

In [ ]:
del smrf_out

### Schofield 

In [ ]:
schofield_snobal_hrrr.hvplot(label='HRRR thermal').opts(**HV_PLOT_OPTS, title='Schofield Pass') * \
schofield_snobal.hvplot(label='SMRF thermal') * \
(schofield_snobal_hrrr - schofield_snobal).hvplot(label='HRRR - SMRF')

In [ ]:
cloud_diff = (schofield_snobal_hrrr_cloud.rename('cloud_factor') - schofield_snobal_cloud).assign_attrs(units='%') * 100

(schofield_snobal_hrrr - schofield_snobal).hvplot(label='HRRR - SMRF therm').opts(**HV_PLOT_OPTS, title='Schofield Pass') * \
cloud_diff.hvplot(label='HRRR - SMRF clouds')

### Butte

In [ ]:
butte_snobal_hrrr.hvplot(label='HRRR thermal').opts(**HV_PLOT_OPTS, title='Butte') * \
butte_snobal.hvplot(label='SMRF thermal') * \
(butte_snobal_hrrr - butte_snobal).hvplot(label='HRRR - SMRF')

In [ ]:
cloud_diff = (butte_snobal_hrrr_cloud.rename('cloud_factor') - butte_snobal_cloud).assign_attrs(units='%') * 100

(butte_snobal_hrrr - butte_snobal).hvplot(label='HRRR - SMRF therm').opts(**HV_PLOT_OPTS, title='Butte') * \
cloud_diff.hvplot(label='HRRR - SMRF clouds')

In [ ]:
taylor_snobal_hrrr.hvplot(label='HRRR thermal').opts(**HV_PLOT_OPTS) * taylor_snobal.hvplot(label='SMRF thermal') 

### Irwin

In [ ]:
irwin_snobal_hrrr.hvplot(label='HRRR thermal').opts(**HV_PLOT_OPTS, title='Irwin') * \
irwin_snobal.hvplot(label='SMRF thermal') * \
(irwin_snobal_hrrr - irwin_snobal).hvplot(label='HRRR - SMRF')

In [ ]:
cloud_diff = (irwin_snobal_hrrr_cloud.rename('cloud_factor') - irwin_snobal_cloud).assign_attrs(units='%') * 100

(irwin_snobal_hrrr - irwin_snobal).hvplot(label='HRRR - SMRF therm').opts(**HV_PLOT_OPTS, title='Irwin') * \
cloud_diff.hvplot(label='HRRR - SMRF clouds')

### Rolling (daily) 

In [ ]:
(schofield_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Schofield').opts(**HV_PLOT_OPTS) * \
schofield_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Schofield') + \
butte_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Butte').opts(**HV_PLOT_OPTS) * \
butte_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Butte') + \
irwin_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Irwin').opts(**HV_PLOT_OPTS) * \
irwin_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Irwin')).cols(1)

In [ ]:
taylor_snobal_hrrr.rolling(time=24, center=True).mean().hvplot(label='HRRR Taylor').opts(**HV_PLOT_OPTS)* \
taylor_snobal.rolling(time=24, center=True).mean().hvplot(label='SMRF Taylor')

In [ ]:
client.shutdown()